In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.40)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.45 sec - Loss 0.062100 - ACC 57.25% - ACC Mean 57.25% - AUC 60.00% - AUC Mean 60.00% - Deter 000
Ite 00050 - 1.07 sec - Loss 0.011731 - ACC 58.00% - ACC Mean 57.88% - AUC 61.10% - AUC Mean 60.91% - Deter 015
Ite 00085 - 0.97 sec - Loss 0.007001 - ACC 58.20% - ACC Mean 57.92% - AUC 62.41% - AUC Mean 60.93% - Deter 050
Early stop ite 85, rollback to correction of ite 35, whith acc of 60.2% and auc of 63.57%
OUR METHOD RUN 1/10 - acc: 60.2% +- 0.0% - auc: 63.57% +- 0.0%
Ite 00000 - 1.07 sec - Loss 0.125068 - ACC 59.50% - ACC Mean 59.50% - AUC 63.05% - AUC Mean 63.05% - Deter 000
Ite 00050 - 1.07 sec - Loss 0.045111 - ACC 57.60% - ACC Mean 58.06% - AUC 60.30% - AUC Mean 61.21% - Deter 025
Ite 00075 - 1.07 sec - Loss 0.032905 - ACC 56.40% - ACC Mean 57.62% - AUC 59.92% - AUC Mean 61.02% - Deter 050
Early stop ite 75, rollback to correction of ite 25, whith acc of 59.95% and auc of 63.37%
OUR METHOD RUN 2/10 - acc: 59.95% +- 0.0% - auc: 63.37% +- 0.0%
Ite 00000 - 1.09 sec - Los

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 59.1% +- 0.0% - auc: 61.8% +- 0.0%
MEAN - acc: 58.35% +- 0.0% - auc: 62.16% +- 0.0%
KNN - acc: 55.5% +- 0.0% - auc: 58.43% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:41<00:00, 119.26it/s]



GAIN RUN 1/10 - acc: 58.45% +- 3.3% - auc: 61.98% +- 3.46%



100%|██████████| 5000/5000 [00:39<00:00, 127.39it/s]



GAIN RUN 2/10 - acc: 59.2% +- 1.86% - auc: 63.26% +- 1.33%



100%|██████████| 5000/5000 [00:39<00:00, 125.26it/s]


...


GAIN RUN 8/10 - acc: 58.9% +- 1.32% - auc: 61.48% +- 1.57%



100%|██████████| 5000/5000 [00:40<00:00, 123.15it/s]



GAIN RUN 9/10 - acc: 59.2% +- 2.3% - auc: 62.31% +- 2.65%



100%|██████████| 5000/5000 [00:42<00:00, 118.41it/s]



GAIN RUN 10/10 - acc: 57.9% +- 2.0% - auc: 61.74% +- 2.26%

GAIN GLOBAL - acc: 59.0% +- 0.67% - auc: 62.65% +- 1.06%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 57.6% +- 0.0% - auc: 61.31% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 57.9% +- 2.13% - auc: 60.63% +- 3.17%

Iteration: 0
Iteration: 1

MISSFOREST RUN 2/10 - acc: 56.45% +- 1.41% - auc: 59.14% +- 0.67%

Iteration: 0
Iteration: 1

MISSFOREST RUN 3/10 - acc: 57.05% +- 2.37% - auc: 59.54% +- 2.26%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 55.5% +- 2.62% - auc: 58.26% +- 2.6%

Iteration: 0
Iteration: 1

MISSFOREST RUN 5/10 - acc: 57.6% +- 2.87% - auc: 61.13% +- 3.24%

Iteration: 0
Iteration: 1

MISSFOREST RUN 6/10 - acc: 56.65% +- 2.41% - auc: 59.8% +- 3.14%

Iteration: 0
Iteration: 1

MISSFOREST RUN 7/10 - acc: 56.6% +- 2.39% - auc: 60.61% +- 2.52%

Iteration: 0
Iteration: 1

MISSFOREST RUN 8/10 - acc: 59.4% +- 2.19% - auc: 62.53% +- 2.64%

Iteration: 0
Iteration: 1

MISSFOREST RUN 9/10 - acc: 56.6% +- 4.5% - auc: 60.23% +- 4.04%

Iteration: 0
Iteration: 1

MISSFOREST RUN 10/10 - acc: 57.95% +- 2.83% - auc: 60.4% +- 3.17%


MISSFOREST GLOBAL - acc: 57.17% +- 1.03% - auc: 60.23% +- 1.1%


## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 58.0% +- 2.06% - auc: 60.43% +- 2.1%
SOFTIMPUTE RUN 2/10 - acc: 57.0% +- 2.59% - auc: 60.96% +- 2.69%
SOFTIMPUTE RUN 3/10 - acc: 57.7% +- 1.48% - auc: 61.82% +- 2.98%
SOFTIMPUTE RUN 4/10 - acc: 59.7% +- 1.82% - auc: 61.62% +- 2.95%
SOFTIMPUTE RUN 5/10 - acc: 58.95% +- 2.67% - auc: 61.81% +- 2.49%
SOFTIMPUTE RUN 6/10 - acc: 57.65% +- 2.14% - auc: 61.3% +- 3.74%
SOFTIMPUTE RUN 7/10 - acc: 59.3% +- 2.69% - auc: 62.69% +- 4.15%
SOFTIMPUTE RUN 8/10 - acc: 58.0% +- 3.04% - auc: 61.02% +- 4.62%
SOFTIMPUTE RUN 9/10 - acc: 58.9% +- 2.49% - auc: 61.88% +- 2.68%
SOFTIMPUTE RUN 10/10 - acc: 58.05% +- 3.31% - auc: 60.32% +- 4.92%

SOFTIMPUTE GLOBAL - acc: 58.32% +- 0.8% - auc: 61.39% +- 0.69%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.4972

SINKHORN RUN 1/10 - acc: 57.1% +- 2.12% - auc: 59.16% +- 3.81%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.5331

SINKHORN RUN 2/10 - acc: 56.45% +- 1.54% - auc: 58.47% +- 1.94%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.4364

SINKHORN RUN 3/10 - acc: 55.55% +- 2.98% - auc: 58.17% +- 2.57%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.7518

SINKHORN RUN 4/10 - acc: 55.55% +- 2.33% - auc: 58.36% +- 1.77%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.6296

SINKHORN RUN 5/10 - acc: 56.65% +- 2.12% - auc: 59.89% +- 1.88%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.5641

SINKHORN RUN 6/10 - acc: 57.45% +- 1.88% - auc: 60.88% +- 1.55%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.4576

SINKHORN RUN 7/10 - acc: 59.25% +- 1.8% - auc: 62.48% +- 2.86%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.6611

SINKHORN RUN 8/10 - acc: 58.25% +- 0.69% - auc: 61.

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 58.75% +- 4.02% - auc: 61.99% +- 4.26%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 56.7% +- 3.01% - auc: 59.65% +- 3.01%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 58.3% +- 1.32% - auc: 60.83% +- 1.53%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 59.15% +- 1.7% - auc: 62.19% +- 2.53%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 56.85% +- 1.82% - auc: 58.78% +- 1.53%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 58.8% +- 2.83% - auc: 61.78% +- 3.02%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 57.35% +- 4.01% - auc: 60.93% +- 3.66%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 57.95% +- 1.76% - auc: 60.54% +- 2.57%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC -